# Data cleaning code
This is the code to clean the MCMF data used in the shiny app. Note: here it doesn't filter for face-to-face programs, but haversine_distances may require that none of the longitude/latitude values are NaN based on your Python version. As such, you may have to filter for face-to-face programs first before cleaning.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import skimpy
import string
import re
from skimpy import clean_columns
from math import radians
from sklearn.metrics.pairwise import haversine_distances

In [2]:
data = pd.read_csv('convert_MCMF_ALL_TIME_DATA.csv', sep='\t')
bus_data = pd.read_csv('CTA_BusStops_Cleaned.csv')

/var/folders/tx/ccgvwwd562sfhw5ybj094vxc0000gn/T/ipykernel_81378/892591545.py:1: DtypeWarning: Columns (14,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('convert_MCMF_ALL_TIME_DATA.csv', sep='\t')


In [3]:
bus_data.head()

,Unnamed: 0,longitude,latitude,the_geom,SYSTEMSTOP,STREET,CROSS_ST,DIR,POS,ROUTESSTPG,OWLROUTES,CITY,PUBLIC_NAM
0,0,-87.747509,41.956892,POINT (-87.74750923800002 41.956892084),15189,CICERO,BERTEAU (north leg),SB,FS,"54,54A",NaN,CHICAGO,Cicero & Berteau
1,1,-87.641786,41.677992,POINT (-87.64178572600002 41.67799153599998),14545,HALSTED,119TH STREET,NB,FS,"8A,108",NaN,CHICAGO,Halsted & 119th Street
2,2,-87.708969,42.012050,POINT (-87.70896851700002 42.012049691000016),15046,KEDZIE,TOUHY,NB,FS,11,NaN,CHICAGO,Kedzie & Touhy
3,3,-87.668916,41.954186,POINT (-87.66891603599998 41.954186386),6280,ASHLAND,IRVING PARK,NB,NS,"9,X9",NaN,CHICAGO,Ashland & Irving Park
4,4,-87.675179,41.832014,POINT (-87.67517869800002 41.83201446999999),4484,ARCHER,DAMEN,SWB,NS,62,N62,CHICAGO,Archer & Damen


In [4]:
def clean_data(data, bus_data):
    # Clean column names
    data = clean_columns(data)
    
    # Remove all programs where min age > 25
    data = data[data["min_age"] < 25]
    
    # Clean category names (program types)
    # Make lowercase
    data.category_name = data.category_name.apply(lambda x: x.lower() if isinstance(x, str) else x)

    def remove_punctuation(text):
        if isinstance(text, str):
            return text.translate(str.maketrans('', '', string.punctuation))
        else:
            return text

    def replace_spaces_with_underscore(text):
        if isinstance(text, str):
            # Replace one or more whitespace characters with a single underscore
            return re.sub(r'\s+', '_', text)
        else:
            return text

    def remove_trailing_underscores(text):
        if isinstance(text, str):
            # Remove any trailing underscores
            text = text.rstrip('_')
            return text
        else:
            return text
        
    # Remove punctuation, replace spaces with underscores, remove trailing underscores for category names
    data.category_name = data.category_name.apply(remove_punctuation)
    data.category_name = data.category_name.apply(replace_spaces_with_underscore)
    data.category_name = data.category_name.apply(remove_trailing_underscores)
    
    # Clean state + city
    data["state"] = data["state"].replace("Illinois", "IL")
    data = data[data.city == "Chicago"]
    
    # Add age range col
    data["age_range"] = data.max_age - data.min_age
    
    # Add distance to nearest bus stop col
    bus_coords = bus_data[['latitude', 'longitude']].to_numpy()
    mcmf_coords = data[['latitude', 'longitude']].to_numpy()
    
    mcmf_in_radians = np.array([[radians(float(x)) for x in coord] for coord in mcmf_coords])
    bus_in_radians = np.array([[radians(float(x)) for x in coord] for coord in bus_coords])
     
    dists = haversine_distances(mcmf_in_radians, bus_in_radians) # Calculate haversine dists then convert to km
    dists_km = dists*6371 # multiply by radius of Earth
    
    min_dist = np.nanmin(dists_km, axis=1) # Minimum distances to each bus stop

    data['km_to_bus_stop'] = min_dist
    
    return data

In [5]:
clean_data(data, bus_data)

56 column names have been cleaned

/var/folders/tx/ccgvwwd562sfhw5ybj094vxc0000gn/T/ipykernel_81378/384819086.py:55: RuntimeWarning: All-NaN slice encountered
  min_dist = np.nanmin(dists_km, axis=1) # Minimum distances to each bus stop


,index_row,id,logo_url,program_name,description,org_name,category_name,capacity,min_age,max_age,...,program_price,geographic_cluster_name,program_pays_participants,program_has_scholarships,program_provides_transportation,latitude,longitude,program_provides_free_food,age_range,km_to_bus_stop
0,"4,568",78752,NaN,BBBS Community Based Mentoring (Year Round),Community Based Mentoring is the traditional B...,Big Brothers Big Sisters,academic_support,2000.0,7,15,...,Free,NEAR WEST SIDE,Not Paid,YES,YES,41.886002,-87.642403,False,8,0.049272
1,"4,569",78752,NaN,BBBS Community Based Mentoring (Year Round),Community Based Mentoring is the traditional B...,Big Brothers Big Sisters,work_career,2000.0,7,15,...,Free,NEAR WEST SIDE,Not Paid,YES,YES,41.886002,-87.642403,False,8,0.049272
2,"4,562",78749,https://cityoflearning-uploads.s3.amazonaws.co...,Youth Mentoring and Academic Supports,Young Men's Educational Network provides mento...,Youth Mentoring and Academic Supports,academic_support,100.0,12,18,...,Free,NORTH LAWNDALE,Not Paid,NO,NO,41.864799,-87.724800,True,6,0.043519
3,"4,563",78749,https://cityoflearning-uploads.s3.amazonaws.co...,Youth Mentoring and Academic Supports,Young Men's Educational Network provides mento...,Youth Mentoring and Academic Supports,music_art,100.0,12,18,...,Free,NORTH LAWNDALE,Not Paid,NO,NO,41.864799,-87.724800,True,6,0.043519
4,"4,557",78746,https://cityoflearning-uploads.s3.amazonaws.co...,Refugee Youth Career Pathways,Refugee Youth Career Pathways (RYCP) provides ...,RefugeeOne,managing_money,15.0,16,24,...,Free,NaN,Not Paid,YES,YES,NaN,NaN,False,8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84873,"77,070",148925,https://cityoflearning-uploads.s3.amazonaws.co...,Medicina Scholars,<p>The Medicina Scholars program is designed t...,Hispanic Center of Excellence at UIC,healthcare,NaN,18,24,...,Free,NEAR WEST SIDE,Unknown_Code,NO,NaN,41.871422,-87.671524,NaN,6,0.007605
84874,"75,634",146348,https://cityoflearning-uploads.s3.amazonaws.co...,Park Cleanup at Kelvyn Park,"<p ""="""">Teens can get service hours through th...",Hermosa Neighborhood Association,helping_your_community,NaN,4,65,...,Free,HERMOSA,Not Paid,NO,NO,41.929092,-87.738152,False,61,0.292268
84875,"75,635",146348,https://cityoflearning-uploads.s3.amazonaws.co...,Park Cleanup at Kelvyn Park,"<p ""="""">Teens can get service hours through th...",Hermosa Neighborhood Association,nature,NaN,4,65,...,Free,HERMOSA,Not Paid,NO,NO,41.929092,-87.738152,False,61,0.292268
84876,"75,638",146350,https://cityoflearning-uploads.s3.amazonaws.co...,Park Cleanup at Kelvyn Park,"<p ""="""">Teens can get service hours through th...",Hermosa Neighborhood Association,helping_your_community,NaN,4,65,...,Free,HERMOSA,Not Paid,NO,NO,41.929092,-87.738152,False,61,0.292268
